In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import pandas as pd
from sklearn.model_selection import train_test_split
from math import floor

import tensorflow as tf

from keras.models import Model
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, \
                    Activation, BatchNormalization, ZeroPadding2D, Dropout

from tensorflow.keras.optimizers import Adam, SGD
import keras.backend as K

import cv2


In [3]:
data_path = "C:\\Users\\Sean\\Desktop\\UCSD Courses\\ECE228 Project\\img_aligned_cropped"

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

x = load_images_from_folder(data_path)

print("DONE")

KeyboardInterrupt: 

In [4]:
print(len(x))

62504


In [7]:
y = pd.read_excel("C:\\Users\\Sean\\Desktop\\UCSD Courses\\ECE228 Project\\img_align_celeba_ALTERED\\celebA_landmarks_front_only.xlsx", "DATA AFTER CROPPING", index_col = 0)
y = y.values.tolist()

x_train, x_test_val, y_train, y_test_val = train_test_split(x, y, test_size=0.2, train_size = 0.8)
x_val, x_test, y_val, y_test = train_test_split(x_test_val, y_test_val, test_size=0.5)

x_train = np.array(x_train)
y_train = np.array(y_train)

x_val = np.array(x_val)
y_val = np.array(y_val)

x_test = np.array(x_test)
y_test = np.array(y_test)


print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print("DONE")

(50003, 128, 128, 3)
(50003, 10)
(6251, 128, 128, 3)
DONE


In [9]:
lr = 8e-5
decay_rate = 1e-4
num_class = 10
img_height, img_width, img_channel = 128, 128, 3

# input layer
input_img = Input(shape=(img_height, img_width, img_channel), name='input_img')


# layer 1
conv1 = Conv2D(20, kernel_size=(5, 5), activation='relu', name='conv1')(input_img)
pool1 = MaxPooling2D(pool_size=(2, 2), name='pool1')(conv1)

# layer 2
conv2 = Conv2D(48, kernel_size=(5, 5), activation='relu', name='conv2')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2), name='pool2')(conv2)

# layer 3
conv3 = Conv2D(64, kernel_size=(3, 3), activation='relu', name='conv3')(pool2)
pool3 = MaxPooling2D(pool_size=(2, 2), name='pool3')(conv3)

# layer 4
conv4 = Conv2D(80, kernel_size=(3, 3), activation='relu', name='conv4')(pool3)

# layer 5
flat = Flatten(name='flat')(conv4)

# landmark detection specific layer
drop1 = Dropout(0.1, name='drop1')(flat)
fc1 = Dense(256, activation='relu', name='fc1')(drop1)
drop2 = Dropout(0.1, name='drop2')(fc1)
out = Dense(num_class, activation='linear', name='out_landmarks')(drop2)

model = Model(inputs=input_img, outputs=out)
opt = Adam(lr=lr, decay=decay_rate)
model.compile(optimizer=opt, loss='mean_squared_error')
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_img (InputLayer)      [(None, 128, 128, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 124, 124, 20)      1520      
                                                                 
 pool1 (MaxPooling2D)        (None, 62, 62, 20)        0         


C:\Users\Sean\anaconda3_ECE228\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


                                                                 
 conv2 (Conv2D)              (None, 58, 58, 48)        24048     
                                                                 
 pool2 (MaxPooling2D)        (None, 29, 29, 48)        0         
                                                                 
 conv3 (Conv2D)              (None, 27, 27, 64)        27712     
                                                                 
 pool3 (MaxPooling2D)        (None, 13, 13, 64)        0         
                                                                 
 conv4 (Conv2D)              (None, 11, 11, 80)        46160     
                                                                 
 flat (Flatten)              (None, 9680)              0         
                                                                 
 drop1 (Dropout)             (None, 9680)              0         
                                                                 
 fc1 (Dens

In [10]:


batch_size = 128
epochs = 10

history = model.fit(
    x_train,
    y_train, 
    batch_size=batch_size, 
    epochs=epochs,
    validation_data=(x_val, y_val)
)

_ = plt.plot(history.history['loss'][1:])
_ = plt.plot(history.history['val_loss'][1:])



Epoch 1/10
236/391 [=================>............] - ETA: 7:34 - loss: 218.5930

KeyboardInterrupt: 

In [ ]:
model.save('landmark_model.h5')